In [6]:
import numpy as np
import math
import numpy.polynomial.polynomial as poly

In [7]:
# info=formatInfo()
# val=24
# pos=info[val][0]
# vel=info[val][1]
# print(angMoment(pos,vel))

In [8]:
def formatInputInfo():
    info=np.loadtxt("/home/soonali/Documents/SoongInput2.txt",dtype=str,delimiter=",")
    time=np.array([float(info[i,0]) for i in range(1,len(info))])
    return time,np.array([([float(info[i][2]),float(info[i][3]),float(info[i][4])], 
                    [float(info[i][5]),float(info[i][6]),float(info[i][7])]) for i in range(1,len(info))])

def formatResultInfo():
    info=np.loadtxt("/home/soonali/Documents/SoongResults.txt",dtype=str,delimiter=",")
    return np.array([[float(info[i][j]) for j in range(2,14)] for i in range(1,len(info))])

def cross(v1:list, v2:list)->list:
    '''Returns the cross product of two 3D vectors'''
    return [(v1[1]*v2[2] - v1[2]*v2[1]),-(v1[0]*v2[2] - v1[2]*v2[0]),(v1[0]*v2[1] - v1[1]*v2[0])]

def dot(v1:list,v2:list)->list:
    return sum([v1[i]*v2[i] for i in range(len(v1))])

def det2x2(m)->float:
    '''Returns the determinant of a 2x2'''
    return m[0][0]*m[1][1]-m[0][1]*m[1][0]

def angMoment(pos,vel):
    res=cross(pos,vel)
    return np.array([round(res[0]/(2*math.pi)*365.2568983,6), round(res[1]/(2*math.pi)*365.2568983,6), round(res[2]/(2*math.pi)*365.2568983,6)])

def getAngle(sin:float,cos:float)->float:
    '''Returns the arcsin of a value as an angle (in radians) in the correct quadrant'''
    return math.atan2(sin,cos) % (math.pi*2)

def getMag(vec):
    return math.sqrt(sum([vec[i]**2 for i in range(len(vec))]))
    
def newton(func,der,init,err): #function, derivative, inital_guess, error_tolerance
    prev=-1
    while abs(init-prev)>=err:
        prev,init=init,init-func(init)/der(init)
    return init


In [9]:
class ODElements:
    def __init__(self, pos, vel, time):
        # constants
        self.k = 0.0172020989484 # au^(3/2)/day
        
        self.time=time
        self.vel=vel
        self.vel=np.array([self.vel[i]/(2*math.pi)*365.2568983 for i in range(3)])
        self.pos=pos
        #print(vel,pos)
        self.angMoment=self.getAngMoment()
        
        self.mu=1
        self.a=self.getSemiMajor() # semi-major axis
        self.e= self.getEcc() # eccentricity
        self.b=None # semi-minor axis
        self.i=self.getInc() # inclination
        self.o=self.getLongAsc() # longitude of ascending node
        self.v=self.getTrueAnom() # true anomaly
        self.w=self.getArgPer() # argument of perihelion
        self.M=self.getMeanAnomaly() # mean anomaly
        self.T=self.getPeriT() # time of perihelion passage T
      
    def getInfo(self):
        '''Returns the info from line'''
        return self.info
    
    def getPos(self):
        return self.pos
    
    def getVel(self):
        return self.vel
    
    def getPosMag(self):
        return math.sqrt(sum([self.pos[i]**2 for i in range(3)]))
    
    def getVelMag(self):
        return math.sqrt(sum([self.vel[i]**2 for i in range(3)]))
    
    def getAngMoment(self):
        '''Returns the angular momentum'''
        pos,vel=self.getPos(),self.getVel()
        res=cross(pos,vel)
        return np.array([res[0], res[1], res[2]])
    
    def getAngMomentMag(self):
        return math.sqrt(sum([self.angMoment[i]**2 for i in range(3)]))
    
    def getSemiMajor(self):
        '''Uses vis-viva'''
        #print(self.getPosMag())
        #print(self.vel)
        #print(self.mu)
        return 1/(2/self.getPosMag() - dot(self.vel,self.vel)/self.mu)
    
    def getEcc(self):
        #print(self.a)
        return math.sqrt(1-dot(self.angMoment, self.angMoment)/(self.mu*self.a))
    
    def getInc(self):
        #print(self.angMoment)
        #print(self.getAngMomentMag())
        return math.acos(self.angMoment[2]/self.getAngMomentMag())*180/math.pi
    
    def getLongAsc(self):
        s=self.angMoment[0]/(self.getAngMomentMag()*math.sin(self.i))
        c=-self.angMoment[1]/(self.getAngMomentMag()*math.sin(self.i))
        return getAngle(s,c)*180/math.pi
    
    def getArgPer(self):
        x,y,z=self.pos[0],self.pos[1],self.pos[2]
        s=z/(self.getPosMag()*math.sin(self.i*math.pi/180))
        c=(x*math.cos(self.o*math.pi/180) + y*math.sin(self.o*math.pi/180))/self.getPosMag()
        U=getAngle(s,c)*180/math.pi
        return (U-self.v)%360
    
    def getTrueAnom(self):
        c=1/self.e*(self.a*(1-self.e**2)/self.getPosMag() - 1)
        s=self.a*(1-self.e**2)/(self.getAngMomentMag() * self.e)*dot(self.pos,self.vel)/self.getPosMag()
        return getAngle(s,c)*180/math.pi
    
    def getPeriT(self):
        n=self.k/(self.a**(3/2))
        return self.time-self.M*math.pi/180/n
    
    def getMeanAnomaly(self):
        s=(self.getPosMag()*math.sin(self.v*math.pi/180))/(self.a*math.sqrt(1-self.e**2))
        c=(self.e+math.cos(self.v*math.pi/180))/(1+self.e*math.cos(self.v*math.pi/180))
        E=getAngle(s,c)
        return (E-self.e*math.sin(E))*180/math.pi
    
    def printError(self, results):
        # EC, QR, IN, OM, W, Tp, N, MA, TA, A, AD, PR,
        print("Semi-major axis:", results[9], self.a, abs(results[9]-self.a)/results[9]*100)
        print("Eccentricity:", results[0], self.e, abs(results[0]-self.e)/results[0]*100)
        print("Inclination:",results[2],self.i,abs(results[2]-self.i)/results[2]*100)
        print("Longitude of Ascending Node:",results[3],self.o,abs(results[3]-self.o)/results[3]*100)
        print("True anomaly:",results[8],self.v,abs(results[8]-self.v)/results[8]*100)
        print("Argument of perihelion:",results[4],self.w,abs(results[4]-self.w)/results[4]*100)
        print("Time of Perihelion Passage T:",results[5],self.T,abs(results[5]-self.T)/results[5]*100)
        print("Mean Anomaly:",results[7],self.M,abs(results[7]-self.M)/results[7]*100)
        #print(od.a, od.e, od.i, od.o, od.v, od.w)
        
    def getElements(self):
        '''a,e,i,o,v,w,T,M'''
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M

def testODElements():
    time,data=formatInputInfo()
    time,info=time[24],data[24]
    results=formatResultInfo()[24]
    od=ODElements(info[0],info[1],time)
    od.printError(results)
    
testODElements()

Semi-major axis: 1.056800055744403 3.2368788582362016 206.29056467603658
Eccentricity: 0.3442331093278357 0.11189968702937521 67.49304933279797
Inclination: 25.1552516656358 15.04818916413854 40.17873736999563
Longitude of Ascending Node: 236.2379806531768 346.7725893987904 46.789516418992115
True anomaly: 158.9559248709949 164.06589743137243 3.2147103447227168
Argument of perihelion: 255.5046142809498 114.75422730309049 55.087219216750384
Time of Perihelion Passage T: 2458158.720849546 2457367.7055299417 0.03217918000555512
Mean Anomaly: 140.4194576216498 160.2396978963978 14.11502409313687


In [10]:
class OD:
    
    def __init__(self):
        self.times,self.data=formatInputInfo()
        self.time,self.info=self.times[0],self.data[0]
        self.od=ODElements(self.info[0],self.info[1],self.time)
        #self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
        
        self.k = 0.0172020989484 #Gaussian gravitational constant  
        self.cAU = 173.144643267 #speed of light in au/(mean solar)day  
        self.eps = math.radians(23.4374) #Earth's obliquity
        
    def getInfoAtTime(self, time:str):
        return
    
    def getElements(self):
        '''a,e,i,o,v,w,T,M'''
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M
    
    def SEL(self, taus, sunR2, rhohat2, coefD):
        '''Scalar Equation of Lagrange'''
        T1,T3,T=taus[0],taus[1],taus[2]
        D0,D21,D22,D23=coefD[0],coefD[1],coefD[2],coefD[3]
        A1=T3/T
        B1=A1/6*(T**2-T3**2)
        A3=-T1/T
        B3=A3/6*(T**2-T1**2)
        A=(A1*D21-D22+A3*D23)/(-D0)
        B=(B1*D21+B3*D23)/(-D0)
        
        E=-2*(dot(rhohat2, sunR2))
        F=dot(sunR2,sunR2)
        
        a=-(A**2+A*E+F)
        b=-self.od.mu*(2*A*B+B*E)
        c=-self.od.mu**2*B**2
        
        coef=[c,0,0,b,0,0,a,0,1]#[1,0,a,0,0,b,0,0,c]
        res=poly.polyroots(coef)

        roots=[]
        for val in res: 
            if np.isreal(val) and np.real(val)>0: roots.append(np.real(val))

        rhos=[A+B/roots[i]**3 for i in range(len(roots))]
        return roots,rhos
        
        

        
def testOD():
    od=OD()
    print("TESTING:\n")
    taus = [-0.15481889055, 0.15481889055, 0.3096377811]  
    sun2 = [-0.2398478458274071, 0.9065739917845802, 0.3929623749770952]  
    rhohat2 = [-0.8518563498182248, -0.2484702599212149, 0.4610892421311239]  
    ds = [-0.0010461861084885213, -0.17297581974209159, -0.17201260125558127, -0.16712421570714076] 
    roots,rhos=od.SEL(taus, sun2, rhohat2, ds)
    print("TEST1")
    print("roots =", roots, "\nrhos =", rhos)
    actroots = [1.014886625023963, 1.2932503440012362, 1.5851855408957922]  
    actrhos = [0.012425430826237482, 0.9753289007273918, 1.386900042701193]  
    errorroots=[]
    errorrhos=[]
    for i in range(len(actroots)):
        errorroots.append((actroots[i]-roots[i])/actroots[i]*100)
        errorrhos.append((actrhos[i]-rhos[i])/actrhos[i]*100)
    print("erroots =", errorroots, "\nrerhos =", errorrhos)
        
    print()
    taus = [-0.1720209895, 0.1720209895, 0.344041979]  
    sun2 = [-0.2683394448727136, 0.8997620807182745, 0.3900022331276332]  
    rhohat2 = [0.052719013914983195, -0.9121555187306237,  0.40643943610469035]  
    ds = [0.0011797570297704812, 0.052586743761143424, 0.05848153743706686,  0.06274019190783499]  
    roots,rhos=od.SEL(taus, sun2, rhohat2, ds)
    print("TEST2")
    print("roots =", roots, "\nrhos =", rhos)
    actroots = [0.7775650888697365, 1.0071727095060097, 1.266484861890734]  
    actrhos = [-0.8448416611523188, -0.01440831314501545, 0.33742943029525474] 
    errorroots=[]
    errorrhos=[]
    for i in range(len(actroots)):
        errorroots.append((actroots[i]-roots[i])/actroots[i]*100)
        errorrhos.append((actrhos[i]-rhos[i])/actrhos[i]*100)
    print("erroots =", errorroots, "\nrerhos =", errorrhos)
    
testOD()

TESTING:

TEST1
roots = [1.0148866250239612, 1.293250344001239, 1.5851855408957929] 
rhos = [0.012425430826227934, 0.9753289007273976, 1.3869000427011935]
erroots = [1.7503007681851243e-13, -2.232034870444734e-13, -4.202245084816129e-14] 
rerhos = [7.684174613579601e-11, -6.033023348457078e-13, -3.202027515877303e-14]

TEST2
roots = [0.7775650888697347, 1.0071727095060117, 1.2664848618907338] 
rhos = [-0.8448416611523293, -0.014408313145011342, 0.3374294302952546]
erroots = [2.284512081145967e-13, -1.984169572371994e-13, 1.753235365115546e-14] 
rerhos = [-1.2352724671794943e-12, 2.8510104894091507e-11, 4.935356514340043e-14]
